In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from typing import Any # import Any from the typing module

# URL of the website
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"

def initialize_driver():
    # Initialize the Chrome driver
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    # Load the page
    driver.get(url)
    time.sleep(5) # wait for the page to load

# Functions to scrape bus routes
def scrape_bus_routes(driver):
    # Scrape bus routes
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Functions to scrape bus details
def scrape_bus_details(driver, url, route_name):
    # Scrape bus details
    try:
        driver.get(url)
        time.sleep(5) # wait for some 5 sec for the page to load

        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5) # wait for some 5 sec for the page to load

            #scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5) # wait for some 5 sec for the page to load

            # Find but details from the availbility
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPATH to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text.strip(),
                    "Bus_Type": bus_type_elements[i].text.strip(),
                    "Departing_Time": departing_time_elements[i].text.strip(),
                    "Duration": duration_elements[i].text.strip(),
                    "Reaching_Time": reaching_time_elements[i].text.strip(),
                    "Star_Rating": star_rating_elements[i].text.strip() if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text.strip(),
                    "Seat_Availability": seat_availability_elements[i].text.strip() if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details

        except Exception as e:
           print(f"Error occurred while scrapping bus details for {url}: {str(e)}")
           return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []
# List to hold all bus details
all_bus_details = []

# Main function to execute the scraping process
def scrap_all_pages():
    for page in range(1, 4): # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5) # wait for some 5 sec for the page to load
                
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            # Iterate over each bus route link and scrape details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                  all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrap_all_pages()

# Create a DataFrame from the scraped data
df_buses_12 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_buses_12.to_csv('Telangana_bus_details.csv', index=False)




In [2]:
df_buses_12

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),04:35,05h 55m,10:30,4.7,500,36 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),05:50,05h 55m,11:45,4.7,450,34 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,FRESHBUS,Electric A/C Seater (2+2),11:05,06h 35m,17:40,4.6,450,38 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,AdSri Durga Malleswari Travels,A/C Sleeper (2+1),23:10,05h 25m,04:35,4.0,665,24 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,Sri KVR Travels,A/C Sleeper (2+1),22:30,06h 30m,05:00,4.6,INR 560,29 Seats available
...,...,...,...,...,...,...,...,...,...,...
1540,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Sri Krishna Travels,A/C Sleeper (2+1),22:30,06h 15m,04:45,2.8,INR 800,21 Seats available
1541,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Sri Krishna Travels,NON A/C Semi Sleeper / Sleeper (2+1),20:00,08h 30m,04:30,2.6,INR 500,34 Seats available
1542,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Mozo travels,A/C Seater / Sleeper (2+1),20:10,06h 50m,03:00,2.1,INR 600,33 Seats available
1543,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...,Vihari Travels,NON A/C Seater/ Sleeper (2+1),20:00,07h 45m,03:45,0,542,28 Seats available
